In [1]:
import urllib3
from bs4 import BeautifulSoup
import requests
import json
import re
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time

In [2]:
os.chdir("../")

In [3]:
base_url = "https://www.imdb.com/"

In [7]:
df = pd.read_csv('./cache/movies.csv', usecols=['id','rating_url'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          250 non-null    int64 
 1   rating_url  250 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.0+ KB


In [11]:
df.head(1)

,id,rating_url
0,0,/title/tt0111161/ratings/?ref_=tt_ov_rt


In [13]:
def get_soup(path_url):
    try:
        full_url = base_url + path_url
        headers = {'User-Agent': 'Thunder Client (https://www.thunderclient.com)'}

        time.sleep(1)
        res = requests.get(full_url, headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    except Exception as e:
        raise e

In [14]:
soup = get_soup('/title/tt0111161/ratings/?ref_=tt_ov_rt')

In [86]:
table = soup.select('table')[0]

In [88]:
rows = table.find_all('tr')

In [98]:
col0 = []
col1 = []
for i in range(1, len(rows)):
    rating_label = [int(re.sub("(\\n)|(\\xa0)|(\s)|(,)", "", r.text)) for r in rows[i].select("div.rightAligned")]
    rating_count = [int(re.sub("(\\n)|(\\xa0)|(\s)|(,)", "", r.text)) for r in rows[i].select("div.leftAligned")]
    col0.extend(rating_label)
    col1.extend(rating_count)

rating_dist = pd.DataFrame({
    "rating":col0,
    "vote_count": col1
})
rating_dist['dist2'] = rating_dist['vote_count']/sum(rating_dist['vote_count'])

In [100]:
rating_dist

,rating,dist,vote_count,dist2
0,10,55.1,1423241,0.550955
1,9,25.6,661104,0.255922
2,8,11.7,301338,0.116652
3,7,3.7,96058,0.037185
4,6,1.1,28951,0.011207
5,5,0.6,15095,0.005843
6,4,0.3,7239,0.002802
7,3,0.2,5365,0.002077
8,2,0.2,4948,0.001915
9,1,1.5,39885,0.015440


In [72]:
table = soup.select('table')[1]

In [85]:
rows = table.find_all('tr')
headers = [h.text for h in rows[0].select("div.tableHeadings")]

col0 = []
col1 = []
col2 = []
col3 = []
for i in range(1, len(rows)):
    group = [r.text for r in rows[i].select("div.leftAligned")]
    rating_label = [float(re.sub("(\\n)|(\\xa0)|(\s)|(,)", "", r.text)) for r in rows[i].select("div.bigcell")]
    rating_count = [int(re.sub("(\\n)|(\\xa0)|(\s)|(,)", "", r.text)) for r in rows[i].select("div.smallcell")]
    col0.extend(group * len(headers))
    col1.extend(headers)
    col2.extend(rating_label)
    col3.extend(rating_count)

pd.DataFrame({
    "group": col0,
    "demographic":col1,
    "rating": col2,
    "count": col3
})

,group,demographic,rating,count
0,All,All Ages,9.3,2583224
1,All,<18,9.3,1350
2,All,18-29,9.3,377141
3,All,30-44,9.3,998457
4,All,45+,9.1,261242
5,Males,All Ages,9.3,1460375
6,Males,<18,9.4,1000
7,Males,18-29,9.4,296537
8,Males,30-44,9.3,827381
9,Males,45+,9.1,211491
